# Проект: Исследование надежности заемщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение, количество детей клиента, его заработная плата и цель кредита на факт погашения кредита в срок.  

Входные данные от банка — статистика о платёжеспособности клиентов.Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.  

**Цели исследования** - проверка наличия свзей между возвратом кредита в срок и наличием детей, семейным положением, уровнем дохода и целью получения кредита.

Импортируем библиотеку

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

Выведем датафрейм

In [2]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Выведим основную информацию о датафрейме с помощью метода `info()`

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

**Выведим количество пропущенных значений для каждого столбца.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаете на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработайте значения в этом столбце: замените все отрицательные значения положительными с помощью метода `abs()`.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

**Для каждого типа занятости выведите медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть.

**Выведите перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

**В столбце `children` есть два аномальных значения. Удалите строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Ещё раз выведите перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

**Заполните пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Убедитесь, что все пропуски заполнены. Проверьте себя и ещё раз выведите количество пропущенных значений для каждого столбца с помощью двух методов.**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Замените вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Обработайте неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру.**

In [14]:
data['education'] = data['education'].str.lower()

**Выведите на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалите их.**

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

**На основании диапазонов, указанных ниже, создайте в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Выведите на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используйте собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучите данные в столбце `purpose` и определите, какие подстроки помогут вам правильно определить категорию.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Исследуйте данные и ответьте на вопросы

**Есть ли зависимость между количеством детей и возвратом кредита в срок?**

Создадим функцию для обработки интересующих нас категорий

In [22]:
def func_pivot(column_name):
    data_pivot = data.pivot_table(index=column_name, values='debt', aggfunc=['count', 'sum', 'mean'])
    data_pivot = data_pivot.rename(columns={'count':'Всего кредитополучателей', 'sum':'Всего должников', 'mean':'Доля должников', 'debt':''})
    data_pivot['Доля должников'] = data_pivot['Доля должников'].apply(lambda x: f"{x*100:.1f}%")
    
    return data_pivot

Вызовем функцию для обработки зависимости между количеством детей и возвратом кредита в срок

In [23]:
data_pivot_children_debt = func_pivot(data['children'])
data_pivot_children_debt

,Всего кредитополучателей,Всего должников,Доля должников
,,,
children,,,
0,14091,1063,7.5%
1,4808,444,9.2%
2,2052,194,9.5%
3,330,27,8.2%
4,41,4,9.8%
5,9,0,0.0%


* 0 - нет задолженности по кредиту
* 1 - есть задолженность по кредиту

**Вывод:** 
Категории граждан у которых пятеро детей, нет ни одной задолженности по кредитам, но по такой выборке нельзя делать выводы, поскольку выборка не презентативна.  

По хорошему можно объединить категорию трое детей и более, но и так видно, что картина от этого практически не изменится (4+27)/(37+303+9) = 0.088*100 = 8.8%. Меньше всего задолженностей в категории у которых ни одного ребенка (8.2%), у других категорий эта доля выше на 1-2%.

**Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Вызовем функцию для обработки зависимости между семейным положением и возвратом кредита в срок

In [24]:
data_pivot_family_debt = func_pivot(data['family_status'])
data_pivot_family_debt

,Всего кредитополучателей,Всего должников,Доля должников
,,,
family_status,,,
Не женат / не замужем,2796,273,9.8%
в разводе,1189,84,7.1%
вдовец / вдова,951,63,6.6%
гражданский брак,4134,385,9.3%
женат / замужем,12261,927,7.6%


**Вывод:** Больше всего задолженностей у двух категорий не женат / не замужем	(10.8%) и в гражданском браке (10.3%), меньше всего задолженностей у категории в разводе (7.6%) и категории вдовец / вдова (7.1%).  

Разница между этими категоряими составляет 2-3%. Возможно это связано со средним возрастом людей в этих категориях, в категориях не женат / не замужем и в гражданском браке предположительно люди более молодого возраста, это влечет за собой в какой-то мере безответственность и недостаток жизненного опыта, в отличие от категорий в разводе и вдовец / вдова, где предположительно находятся люди более зрелого возраста у которых и опыта больше и ответственности.

**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Вызовем функцию для обработки зависимости между уровнем дохода и возвратом кредита в срок

In [25]:
data_pivot_income_debt = func_pivot(data['total_income_category'])
data_pivot_income_debt

,Всего кредитополучателей,Всего должников,Доля должников
,,,
total_income_category,,,
A,25,2,8.0%
B,5014,354,7.1%
C,15921,1353,8.5%
D,349,21,6.0%
E,22,2,9.1%


**Напомним категории дохода:**
* А - более 1 млн руб.
* B - 200 001 - 1 000 000 руб.
* С - 50 001 - 200 000 руб.
* D - 30 001 - 50 000 руб.
* E - до 30 000 руб.

**Вывод:** Как видно из таблицы, наилучший показатель возврата кредита в срок у категории D, наихудший показатель у категории E, но у категории E очень малая выборка, для выводов не достаточно.  

Наиболее презентативная выборка у категорий C и B, если сравнивать эти две категории, то мы увидим разницу почти в 2%, граждане категории B являются более надежными кредиторами по сравнению с гражаднами категории C.  

Возможно это связано с тем, что в категории C достаточно большой диапазон заработных плат относительно среднего достатка и у тех граждан у которых доход ближе к отметке 50 000 руб. показатель возврата кредита в срок будет хуже, чем у тех, у кого доход ближе к 200 000 руб.

**Как разные цели кредита влияют на его возврат в срок?**

Вызовем функцию для обработки зависимости между целями кредита  и возвратом кредита в срок

In [26]:
data_pivot_purpose_debt = func_pivot(data['purpose_category'])
data_pivot_purpose_debt

,Всего кредитополучателей,Всего должников,Доля должников
,,,
purpose_category,,,
операции с автомобилем,4279,400,9.3%
операции с недвижимостью,10751,780,7.3%
получение образования,3988,369,9.3%
проведение свадьбы,2313,183,7.9%


**Вывод:** Самыми надежными заемщиками являются те, кто берет кредиты на проведение свадьбы и покупку недвижимости.  

Наихудший показатель возврата кредитов у тех, кто берет кредиты на автомобиль и получение образования.

**Приведите возможные причины появления пропусков в исходных данных.**

*Ответ:* Человеческий фактор, ошибки в выгрузке таблиц, пропуски были оставлены с какой-то целью

**Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.**

*Ответ:* У количественных переменных значения могут сильно выбиваются среди большинства, поэтому пропуски лучше заполнять медианными значениями.

**Общий вывод**  

В ходе работы над проектом, была проведена предобработка данных - устранены пропуски, некорректные значения, аномалии, явные и неявные дубликаты. Часть из них предположительно была ошибками ввода данных, часть - некорректно собранными или обработанными данными. Были добавлены столбцы с выборкой по категориям, для более тщательного анализа.

Были проанализованы зависимости между данными о заемщике и возвратом кредита в срок.  

Больших расхождений по категориям выявить не удалось, в среднем расхождения по категориям между надежными и ненадежными заемщиками составляют 2-3%. Чаще всего возвращают в срок кредиты на операции с недвижимостью, из всех категорий наиболее надежными заемщиками являются лица не имеющие детей, со среднем доходом или выше. Больше всего задержек по кредитам на автомобили и образование, наименее надежные заемщики - лица у которых два и более ребенка, не состоящих в официальном браке.